In [1]:
import pandas as pd
import numpy as np
import re
df = pd.read_csv("D:\\Academics\\Hackathon\\UIDAI Data HAckathon-2026\\dataset\\api_data_aadhar_demographic.csv")
print(df.head())
print(df.shape)
print(df.columns)

         date           state    district  pincode  demo_age_5_17  \
0  01-03-2025   Uttar Pradesh   Gorakhpur   273213             49   
1  01-03-2025  Andhra Pradesh    Chittoor   517132             22   
2  01-03-2025         Gujarat      Rajkot   360006             65   
3  01-03-2025  Andhra Pradesh  Srikakulam   532484             24   
4  01-03-2025       Rajasthan     Udaipur   313801             45   

   demo_age_17_  
0           529  
1           375  
2           765  
3           314  
4           785  
(2071700, 6)
Index(['date', 'state', 'district', 'pincode', 'demo_age_5_17',
       'demo_age_17_'],
      dtype='object')


In [2]:
# Fix missing / out-of-range values 

for col in ['demo_age_5_17', 'demo_age_17_']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df[df[col] >= 0]
    df[col] = df[col].fillna(df[col].median())

In [3]:
#Normalized Data Form

def clean_text(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip()
    x = re.sub(r"\s+", " ", x)
    return x

def is_numeric_junk(x):
    if pd.isna(x):
        return True
    return str(x).isdigit()

def normalize_name(x):
    x = str(x).lower().strip()
    x = re.sub(r"[^a-zA-Z ]", "", x)
    x = re.sub(r"\s+", " ", x)
    return x.title()

In [4]:
print(df)

               date           state        district  pincode  demo_age_5_17  \
0        01-03-2025   Uttar Pradesh       Gorakhpur   273213             49   
1        01-03-2025  Andhra Pradesh        Chittoor   517132             22   
2        01-03-2025         Gujarat          Rajkot   360006             65   
3        01-03-2025  Andhra Pradesh      Srikakulam   532484             24   
4        01-03-2025       Rajasthan         Udaipur   313801             45   
...             ...             ...             ...      ...            ...   
2071695  29-12-2025     West Bengal  West Midnapore   721212              0   
2071696  29-12-2025     West Bengal  West Midnapore   721420              0   
2071697  29-12-2025     West Bengal  West Midnapore   721424              0   
2071698  29-12-2025     West Bengal  West Midnapore   721426              0   
2071699  29-12-2025     West Bengal         hooghly   712701              0   

         demo_age_17_  
0                 529  
1  

In [5]:
#clean State

df['state'] = df['state'].apply(clean_text)
df['district'] = df['district'].apply(clean_text)

junk_mask = df['state'].apply(is_numeric_junk) | df['district'].apply(is_numeric_junk)
df = df[~junk_mask]

df['state'] = df['state'].apply(normalize_name)
df['district'] = df['district'].apply(normalize_name)

In [6]:
#fix date and Pincode

df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df[~df['date'].isna()]

df['pincode'] = df['pincode'].astype(str)

df = df.drop_duplicates()

In [7]:
print(df)

              date           state     district pincode  demo_age_5_17  \
0       2025-01-03   Uttar Pradesh    Gorakhpur  273213             49   
1       2025-01-03  Andhra Pradesh     Chittoor  517132             22   
2       2025-01-03         Gujarat       Rajkot  360006             65   
3       2025-01-03  Andhra Pradesh   Srikakulam  532484             24   
4       2025-01-03       Rajasthan      Udaipur  313801             45   
...            ...             ...          ...     ...            ...   
1989317 2025-10-12     West Bengal  Murshidabad  742137              0   
1989318 2025-10-12     West Bengal  Murshidabad  742139              0   
1989319 2025-10-12     West Bengal  Murshidabad  742147              1   
1989320 2025-10-12     West Bengal  Murshidabad  742161              1   
1989321 2025-10-12     West Bengal  Murshidabad  742165              0   

         demo_age_17_  
0                 529  
1                 375  
2                 765  
3              

In [8]:
#Location Id

location_master = (
    df[['state', 'district', 'pincode']]
    .drop_duplicates()
    .reset_index(drop=True)
)

location_master['location_id'] = location_master.index + 1

df = df.merge(
    location_master,
    on=['state', 'district', 'pincode'],
    how='left'
)

In [9]:
#fact 
demographic_fact = df.melt(
    id_vars=['date', 'location_id'],
    value_vars=['demo_age_5_17', 'demo_age_17_'],
    var_name='age_group',
    value_name='population'
)

demographic_fact['age_group'] = demographic_fact['age_group'].replace({
    'demo_age_5_17': '5-17',
    'demo_age_17_': '18+'
})

age_group_master = pd.DataFrame({
    'age_group_id': [1, 2],
    'age_group': ['5-17', '18+']
})

demographic_fact = demographic_fact.merge(
    age_group_master,
    on='age_group',
    how='left'
)

demographic_fact = demographic_fact.drop(columns=['age_group'])

In [10]:
#check the data is consistent or not

# Check the missing values
print(demographic_fact.isna().sum())

# check the negative value is present or not
print((demographic_fact['population'] < 0).sum())

# Ensure location IDs are valid
print(demographic_fact['location_id'].nunique() == location_master.shape[0])

date            0
location_id     0
population      0
age_group_id    0
dtype: int64
0
True


In [11]:
#to Export the data as csv
df.to_csv("C:\\Users\\Karan Raj\\Downloads\\demographic\\demographic.csv", index=False)
print("Exported")

Exported
